In [32]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import netCDF4
import scipy
from scipy import stats
def linr(a,b):
    slope, intercept, r_value, p_value, std_err = stats.linregress(a,b)
    return slope
def rms_calc(ser):
    return np.sqrt(np.mean(ser**2))

In [33]:
f = xr.open_dataset(r'C:\Users\lv299\OneDrive\Melbourne-Uni\Week 4\Monday\correction.nc')
ds = f.__xarray_dataarray_variable__
nds = ds.loc[:,5:-5.1,:]

In [34]:
nino = xr.open_dataset(r'C:\Users\lv299\OneDrive\Melbourne-Uni\Week 4\nino3.4.nc')
nino34 = nino.sst.loc['1982-01-01':'2022-02-01']
nino34 = nino34.resample(time ="M").mean(dim="time")
#chazhi_nino34 = nino34.value
nino34month = nino34.resample(time='QS-Dec').mean(dim="time")
ninoDJF=nino34month[4::4]
ninoDJF['time']=ninoDJF['time.year']
ninoDJF=ninoDJF.rename({'time': 'Year'})

# ninoDJF from 1982_12 1 2 to 2021_12 1 2

ds = ninoDJF
climatology_mean = ds.mean("Year")
climatology_std = ds.std("Year")
stand_anomalies = xr.apply_ufunc(
    lambda x, m, s: (x - m) / s,
    ds,
    climatology_mean,
    climatology_std,
)

nino34 = stand_anomalies

nino34 

<xarray.DataArray 'sst' (Year: 40)>
array([ 1.985761  , -0.6588748 , -0.9571033 , -0.481748  ,  1.0650433 ,
        0.62577856, -1.6167519 , -0.04313108,  0.2644623 ,  1.5439421 ,
        0.11678059,  0.10998295,  0.9009185 , -0.8011444 , -0.39681795,
        2.1231494 , -1.3149934 , -1.3846452 , -0.61373633, -0.12783563,
        0.8748333 ,  0.33405834,  0.5883795 , -0.62254685,  0.58369744,
       -1.3995228 , -0.6456683 ,  1.3213552 , -1.2765263 , -0.7767258 ,
       -0.22009206, -0.28910893,  0.58297914,  2.3240397 , -0.27310014,
       -0.8208922 ,  0.8049623 ,  0.5027895 , -0.9886691 , -0.94327855],
      dtype=float32)
Coordinates:
  * Year     (Year) int64 1982 1983 1984 1985 1986 ... 2017 2018 2019 2020 2021

In [45]:
sqnds = np.square(nds.values)
lat = nds['lat']
lon = nds['lon']
time = nds['time']
sqnds = xr.DataArray(sqnds,coords = {'time' : time, 'lat' : lat, 'lon' : lon},dims = ["time", "lat", "lon"])

varsqnds = sqnds.rolling(time = 91, center = True).std(ddof=0).dropna("time")

ValueError: Must pass 2-d input. shape=(14975, 5, 144)

In [38]:
varsqnds.max()

<xarray.DataArray ()>
array(1528.60468921)

In [37]:
varsqnds.min()

<xarray.DataArray ()>
array(0.09512432)